In [1]:
import os
import pandas as pd
from ons import get_sheet_names, detect_id_row, get_headers, add_group, load_data, create_date_column, melt_table

In [2]:
unem01_file = '../../data/raw/unem01-latest.xls'

In [3]:
sheets = get_sheet_names(unem01_file)
sheets

['Note', 'People', 'Men', 'Women']

In [4]:
id_row = detect_id_row(unem01_file)

In [5]:
headers = pd.concat([get_headers(unem01_file, s, id_row).pipe(add_group, s) for s in sheets[1:]])
headers

,age,category,measure_type,group
Dataset identifier code,,,,
MGSC,All aged 16 & over,All,level,People
MGSX,All aged 16 & over,All,rate (%)1,People
YBWF,All aged 16 & over,Up to 6 months,level,People
YBWG,All aged 16 & over,Over 6 and up to 12 months,level,People
YBWH,All aged 16 & over,All over 12 months,level,People
...,...,...,...,...
YBYY,50 and over,Up to 6 months,level,Women
YBZB,50 and over,Over 6 and up to 12 months,level,Women
YBZE,50 and over,All over 12 months,level,Women


In [ ]:
headers_file = '../../data/metadata/labour-market/unem01-codes.csv'
os.makedirs(os.path.dirname(headers_file), exist_ok=True)
headers.to_csv(headers_file)

In [6]:
data = pd.concat([
  load_data(unem01_file, s, id_row).pipe(create_date_column).pipe(melt_table).merge(headers, left_on='variable_name', right_index=True)
  for s
  in sheets[1:]
])

In [7]:
out_file = '../../data/processed/labour-market/unem01.csv'
os.makedirs(os.path.dirname(out_file), exist_ok=True)
data.to_csv(out_file, index=False)